In [4]:
import os 
import numpy as np
import pandas as pd
import matplotlib as plt
import re
from functools import reduce

filename='D:\\PythonLearning\\Scrapy\\PythonProject\Scrapy_Project\\History Data\\football_MexicoApertura2022data.csv'
#input('please key in the file name: ')
print(filename)
df =pd.read_csv(filename)
df=df[df['DetailScore'] != "('link error',)"]
df=df[df['status_match'] == "FT"]
re_char={"'":"",",":"","(":"",")":"",";":" ","-":" "}
df['score']=df['DetailScore'].map(lambda a: (''.join(re_char.get(x, x) for x in str(a))) )
df['MaximumGoal']= df['score'].apply(lambda a : max(map(int,[e for e in re.split("[^0-9]",str(a)) if e !='']),default=0))
df['Over70']= df['score'].apply(lambda a :any(int(item)>70 for item in str(a).split()))
df['Over90']= df['score'].apply(lambda a :any(int(item)>=90 for item in str(a).split()))

df['NOGoalH2']=df['MaximumGoal'].map(lambda a : int(a)<=45)
df['index']=df['round_match'].apply(lambda round: round.split()[1].replace(" ",""))

# #print(df['index'])
df['index']=pd.to_numeric(df['index'])

df_sorted=df.sort_values(by=['index'],ascending=False)
# Add a new column called "HTGoalCount" to df
df_sorted['HTGoal'] = df_sorted['HTResult'].map(lambda a: (''.join(re_char.get(x, x) for x in str(a))))
df_sorted['HTGoalCount']=df_sorted['HTGoal'].map(lambda x: sum(int(y) for y in x if y.isdigit()))



D:\PythonLearning\Scrapy\PythonProject\Scrapy_Project\History Data\football_MexicoApertura2022data.csv


In [5]:
df_sorted['TimeHomeScrore']=df_sorted['TimeHomeScrore'].map(lambda a: (''.join(re_char.get(x, x) for x in str(a))) )
df_sorted['HomeScore']=df_sorted['TimeHomeScrore'].str.count('\d+')
df_sorted['TimeAwayScrore']=df_sorted['TimeAwayScrore'].map(lambda a: (''.join(re_char.get(x, x) for x in str(a))) )
df_sorted['AwayScore']=df_sorted['TimeAwayScrore'].str.count('\d+')

df_sorted['HomeWin'] = df_sorted['HomeScore'] > df_sorted['AwayScore']
df_sorted['AwayWin'] = df_sorted['AwayScore'] > df_sorted['HomeScore']
df_sorted['Draw']=df_sorted['AwayScore'] == df_sorted['HomeScore']
df_sorted['HT_NoGoal']=df_sorted['HTGoalCount']<1


In [6]:
match=df_sorted #[df_sorted['Over70']==True]

matchover70byround=match.groupby(['round_match','index','Over70']).count()['FTResult'].unstack().fillna(0)
matchNoGoalH2byround=match.groupby(['round_match','index','NOGoalH2']).count()['FTResult'].unstack().fillna(0)
matchover70byround.sort_values(by='index')
#print(matchNoGoalH2byround.sort_values(by='index'))
#matchNoGoalH2byround.info()
over70=match[['round_match','index','Over70']].groupby(['index','Over70']).count().unstack().fillna(0)
over90=match[['round_match','index','Over90']].groupby(['index','Over90']).count().unstack().fillna(0)
GoalH2=match[['round_match','index','NOGoalH2']].groupby(['index','NOGoalH2']).count().unstack().fillna(0)
HomeWin=match[['round_match','index','HomeWin']].groupby(['index','HomeWin']).count().unstack().fillna(0)
AwayWin=match[['round_match','index','AwayWin']].groupby(['index','AwayWin']).count().unstack().fillna(0)
Draw=match[['round_match','index','Draw']].groupby(['index','Draw']).count().unstack().fillna(0)
HT_NoGoal=match[['round_match','index','HT_NoGoal']].groupby(['index','HT_NoGoal']).count().unstack().fillna(0)
#over70['key1']=over70.index
over70['Over70']='Over70'
over90['Over90']='Over90'
#GoalH2['key2']=GoalH2.index
GoalH2['NOGoalH2']='NOGoalH2'
HomeWin['HomeWin']='HomeWin'
AwayWin['AwayWin']='AwayWin'
Draw['Draw']='Draw'
HT_NoGoal['HT_NoGoal']='HT_NoGoal'
# print(over70.sort_values(by='index'))
# print(GoalH2.sort_values(by='index'))
result=pd.concat(
                [   
                    over70.sort_values(by='index'),
                    GoalH2.sort_values(by='index'),
                    HomeWin.sort_values(by='index'),
                    AwayWin.sort_values(by='index'),
                    Draw.sort_values(by='index'),
                    over90.sort_values(by='index'),
                    HT_NoGoal.sort_values(by='index')
                ],axis=1,join="inner")
print(result)
# export to excel file
result.to_excel('1_check.xlsx')




      round_match       Over70 round_match       NOGoalH2 round_match        
            False True               False True                 False True   
index                                                                        
1               5    4  Over70         7.0  2.0  NOGoalH2           7    2  \
2               3    6  Over70         6.0  3.0  NOGoalH2           6    3   
3               3    6  Over70         7.0  2.0  NOGoalH2           5    4   
4               7    2  Over70         3.0  6.0  NOGoalH2           5    4   
5               5    4  Over70         5.0  4.0  NOGoalH2           7    2   
6               6    3  Over70         5.0  4.0  NOGoalH2           4    5   
7               3    6  Over70         8.0  1.0  NOGoalH2           1    8   
8               3    6  Over70         7.0  2.0  NOGoalH2           7    2   
9               4    5  Over70         9.0  0.0  NOGoalH2           7    2   
10              5    4  Over70         6.0  3.0  NOGoalH2       